In [14]:
##################################################################
# This script is for generating map.json from 
# original label_v2.json file by standardizing predefined product_type
#
# Please make sure the input file path and type is correct
#
# 
# Author: Luis Lin
# Date: June 27, 2022
#################################################################

In [15]:
import json
from typing import List
import spacy

In [16]:
f = open("/Users/luis/Documents/GitHub/2022Summer/AI-Oriented-Recommendation-System/PreprocessingData/tools/json_files/labels_v2.json")
label = json.load(f)
pt = label.get("product_type")
st = label.get("sub_product_type")
nlp = spacy.load("en_core_web_sm")

'''
input: list, e.g. ['t', '-', 'shirt'], ['high', '-', 'rise', 'shoe']
function: deal with the list with '-' punct
output: "t-shirt" "high-rise shoe" "a-b-c color" 
'''
def check_and_concat_punct(temp:List[str]) -> str:
    temp_string = ""
    if temp.__contains__('-') and len(temp) == 3:
        temp_string = ''.join(temp)
    elif temp.__contains__('-') and len(temp) > 3:
        
        pos = temp.index('-')
        #if there is wrong position of '-'
        if pos < 1:
            temp_string = ' '.join(temp)
            return temp_string
        
        puct_string = temp[pos - 1] + temp[pos] + temp[pos + 1]
        rest_string = ' '.join(temp[pos + 2:])
        temp_string = puct_string + ' '+ rest_string
    else:
        temp_string = ' '.join(temp)
    
    return temp_string 

'''
input: original string e.g. 'scandals', 'jackets/coats'
funct: clean the string to the original words and put to list
output: ['scandal'], ['jacket', 'coats'] 
'''
def lemma_string(original_string:str) -> List:
    doc = nlp(original_string.lower())
    temp = []
    for token in doc:
        if token.text == '&' or token.text == 'and' or token.text == '/' or token.text == ',':
            continue
        if token.text == "glasses" or token.text == 'booty' or token.text == "sunglasses" or token.text == "earrings": #some of them should be represented in plural format
            temp.append(token.text)
        else:
            temp.append(token.lemma_)
    return temp

In [17]:
sub_cat_all_keys = [key for key in st[0].keys()]
sub_cat_shoes_keys = sub_cat_all_keys[0:4]
sub_cat_tops_keys= sub_cat_all_keys[4:7]
sub_cat_other_clothing_keys = sub_cat_all_keys[7:8] + sub_cat_all_keys[13:14]
sub_cat_bottoms_keys = sub_cat_all_keys[8:13]
sub_cat_acc_keys = sub_cat_all_keys[14: 19]
sub_cat_beauty_keys = sub_cat_all_keys[19: ]

In [18]:
all_main = {key for key in pt[0].keys()}
#print("First: \n",  all_main)
#set
all_sub_main = set()
#dict { 'shoes': {(all shoe types whatever it is the 2nd or 3rd class label)} }
products_to_all = {}
#dict { 'shoes': 1, ''}
main_categories_map_to_num = {}
#dict { 'heels': 1, 't-shirt': } specific item maps to the number
specific_products_map_to_num = {}
pt_index = 0
for key in pt[0].keys():
    product_name_2nd_list = pt[0].get(key)
    pt_key = key
    pt_value_set = set()
    
    for item_string in product_name_2nd_list:
        temp = lemma_string(item_string)
        temp_string = check_and_concat_punct(temp)

        #update set
        all_sub_main.add(temp_string)
        #update the value in the pair[]
        pt_value_set.add(temp_string)
    
    #main_to_sub map
    products_to_all.update({pt_key:pt_value_set})
    #product_to_num map
    pt_index += 1
    main_categories_map_to_num.update({pt_key:pt_index})

main_categories_map_to_num.update({"other" : 8})
main_categories_map_to_num.update({"unknown" : 0})
#if key in [scandals, sneakers, heels, boots], or [boots, shirts, sweaters, jackets, coats]
#print("Second: all main_categories\n", all_sub_main)

all_sub = set()
for key in st[0].keys():
    product_name_3rd_list = st[0].get(key) # get list of []
    for item_string in product_name_3rd_list: 
        #loop all item_string in order
        temp = lemma_string(item_string)
        temp_string = check_and_concat_punct(temp)
        temp_list = lemma_string(key)
        temp_string2 = check_and_concat_punct(temp_list)
        all_sub.add(temp_string)
        all_sub.add(temp_string2)
#print("Third: all sub catrgories\n", all_sub)

In [19]:
'''
input: the list of all sub_categories, the key of the main_categories
function:put all sub_categories to the relative the main_categories set
output: none
'''
def update_products_to_all(string_list : List, main_key: str)->None:
    assert len(string_list) > 0
    assert products_to_all.get(main_key) is not None
    for item_string in string_list:
        temp = lemma_string(item_string)
        temp_string = check_and_concat_punct(temp) 
        products_to_all.get(main_key).add(temp_string)

In [20]:
#add all sub categories to the main 8 categories.
for key in sub_cat_all_keys:#loop all keys in sub_main_categories, 
    if key in sub_cat_tops_keys:#if the key belongs to tops
        product_name_3rd_list = st[0].get(key)
        update_products_to_all(product_name_3rd_list + sub_cat_tops_keys, 'tops')
    elif key in sub_cat_shoes_keys:
        product_name_3rd_list = st[0].get(key)
        update_products_to_all(product_name_3rd_list + sub_cat_shoes_keys, 'shoes')
    elif key in sub_cat_bottoms_keys:
        product_name_3rd_list = st[0].get(key)
        update_products_to_all(product_name_3rd_list + sub_cat_bottoms_keys, 'bottoms')
    elif key in sub_cat_other_clothing_keys:
        product_name_3rd_list = st[0].get(key)
        update_products_to_all(product_name_3rd_list + sub_cat_other_clothing_keys, 'other_clothing')
    elif key in sub_cat_acc_keys:
        product_name_3rd_list = st[0].get(key)
        update_products_to_all(product_name_3rd_list + sub_cat_acc_keys, 'accessories')
    elif key in sub_cat_beauty_keys:
        product_name_3rd_list = st[0].get(key)
        update_products_to_all(product_name_3rd_list + sub_cat_beauty_keys, 'beauty') 

In [21]:
for key in products_to_all.keys():
    products_to_all.update({key: list(products_to_all.get(key))})

In [22]:
'''
Build the hashmap all products has mapped to the number of categories
'''
specific_products_map_to_num = {}
for key in products_to_all.keys():
    map_number_of_this_key = main_categories_map_to_num.get(key)
    assert products_to_all.get(key) is not None
    #include the main_categories to number;
    specific_products_map_to_num.update({key: map_number_of_this_key})
    for item in products_to_all.get(key):
        specific_products_map_to_num.update({item: map_number_of_this_key})

In [23]:
products_to_all

{'shoes': ['sneaker',
  'boot',
  'slip-on sneaker',
  'rain boot',
  'flat shoe',
  'snowshoe',
  'mule',
  'heel boot',
  'ankle boot',
  'chelsea boot',
  'heel',
  'espadrille',
  'ice gripper boot',
  'classic sneaker',
  'canvas sneaker',
  'skate shoe',
  'high top sneaker',
  'wedge heel',
  'platform heel',
  'sandal',
  'chunky sandal',
  'oxford loafer',
  'platform',
  'lace-up boot',
  'stilettos',
  'clog',
  'falt sandal',
  'wedge',
  'ankle strap heel',
  'winter boot',
  'platform boot',
  'slide',
  'tall boot',
  'heel wedge',
  'jelly',
  'pump',
  'kitten heel',
  'slipper',
  'flip flop',
  'waterproof boot'],
 'tops': ['bra',
  'button-up shirt',
  'sleeveless hooodie',
  'tee',
  'kimono',
  'bomber jacket',
  'denim jacket',
  'sleeveless tee',
  'crop hoodie',
  'poncho',
  'waistcoat',
  'camisole',
  'hoodie',
  'technical sport jacket',
  'croptee',
  'polo shirt',
  'sweater vests camis',
  'parka',
  'office shirt',
  't',
  'button-down shirt',
  'quilt

In [24]:
#save files
with open("json_files/product_to_all.json", "w") as f1:
    json.dump(products_to_all, f1)
#parsed = json.dumps(products_to_all, indent=4)
with open("json_files/main_categories_to_num.json","w") as f2:
    json.dump(main_categories_map_to_num,f2)

with open("json_files/specific_product_map_to_num.json", "w") as f3:
    json.dump(specific_products_map_to_num, f3)